In [ ]:
from typing import List
mat = [[8, 9], [10, 33]]

(a, b), (c, d) = mat[0], mat[1]
print(a, b, c, d)
det = a * d - b * c
assert det != 0, "det cannot be zero"
det = 2
inv_mat = [[d, -b], [-c, a]]
print([[item / det for item in inv_mat[j]] for j in range(2)])


def matrix_mul(mat1: List[List], mat2: List[List]) ->List[List]:
    assert len(mat1[0]) == len(mat2), "mat1's lines must be equal to mat2's rows"
    result = []
    for r in mat1:
        res_row = []
        for l in zip(*mat2):
            temp = [i * j for i, j in zip(r, l)]
            res_row.append(sum(temp))
        result.append(res_row)
    
    return result

print(matrix_mul(mat, [[5], [6]]))


In [ ]:
from concurrent.futures import ProcessPoolExecutor
import math
from multiprocessing import Pool, cpu_count


p_kn = []
p_kn.append(list(map(math.log, range(1, 10000))))

for i in range(3, 2):
    print(i)
# with Pool(cpu_count()) as p:
#     p_kn.append(list(p.map(math.log, range(1, 10000))))

# with ProcessPoolExecutor() as executor:
#     p_kn.append(list(executor.map(math.log, range(1, 10000))))

In [ ]:
import os
import time
import heapq as hq
import numpy as np
import pandas as pd
from tqdm import tqdm
import pdb

class Collaborative_Filtering:
    def __init__(self, save_topk: int, min_common_user: int):
        self.movie_similary = dict()
        self.save_topk = save_topk
        self.min_common_user = min_common_user
        self.data = None
        self.group_dict = dict()    # 每部电影被哪些用户看过
        self.score_dict = dict()    # 每部电影被哪些用户看过相应的评分
        self.user_dict = dict()     # 每个用户看过哪些电影


    def cosin_similary(self, m1, m2):
        users1 = self.group_dict[m1]
        users2 = self.group_dict[m2]
        scores1 = self.score_dict[m1]
        scores2 = self.score_dict[m2]

        # 构造 user -> rating 映射
        map1 = {u: r for u, r in zip(users1, scores1)}
        map2 = {u: r for u, r in zip(users2, scores2)}

        # 共同评分用户
        common_users = list(set(map1.keys()) & set(map2.keys()))
        if len(common_users) < self.min_common_user:
            return None

        # 固定顺序（关键修复点）
        common_users.sort()

        # 计算用户均值（仅在共同用户集合上）
        user_mean = {
            u: (map1[u] + map2[u]) / 2
            for u in common_users
        }

        # 构造“去用户均值”的评分向量
        v1 = np.array([map1[u] - self.user_mean[u] for u in common_users], dtype=np.float64)
        v2 = np.array([map2[u] - self.user_mean[u] for u in common_users], dtype=np.float64)

        norm1 = np.linalg.norm(v1)
        norm2 = np.linalg.norm(v2)
        if norm1 == 0 or norm2 == 0:
            return None

        sim = np.dot(v1, v2) / (norm1 * norm2)
        return sim

    def keep_topk_sim(self, heap_m, insert_m, sim):
        small_heap = self.movie_similary[heap_m]
        small_heap.append((sim, insert_m))
        # if len(small_heap) < self.save_topk:
        #     hq.heappush(small_heap, (sim, insert_m))
        # else:
        #     if sim > small_heap[0][0]:
        #         hq.heapreplace(small_heap, (sim, insert_m))

    def compute_user_mean(self):
        user_sum = {}
        user_cnt = {}

        for movie, users in self.group_dict.items():
            scores = self.score_dict[movie]
            for u, r in zip(users, scores):
                user_sum[u] = user_sum.get(u, 0.0) + r
                user_cnt[u] = user_cnt.get(u, 0) + 1

        # 用户平均评分
        self.user_mean = {
            u: user_sum[u] / user_cnt[u]
            for u in user_sum
        }


    def train(self, data: pd.DataFrame):
        movie = list(set(data["movieId"]))
        for m in movie:
            self.movie_similary.update({m: []})

        for row in data.itertuples(index=False):
            if row.userId not in self.user_dict:
                self.user_dict.update({row.userId: [row.movieId]})
            else:
                self.user_dict[row.userId].append(row.movieId)
            
            if row.movieId not in self.group_dict:
                self.group_dict.update({row.movieId: [row.userId]})
                self.score_dict.update({row.movieId: [row.rating]})
            else:
                self.group_dict[row.movieId].append(row.userId)
                self.score_dict[row.movieId].append(row.rating)
        
        self.compute_user_mean()

        for i1, m1 in enumerate(movie):
            for i2 in range(i1 + 1, len(movie)):
                m2 = movie[i2]

                sim = self.cosin_similary(m1, m2)
                if sim is not None:
                    self.keep_topk_sim(m1, m2, sim)
                    self.keep_topk_sim(m2, m1, sim)
            
            percent = i1 / len(movie) * 100
            print(f"\rProgress: {percent:6.2f}%", end="")



    def predict(self, user_id, movie_id):
        user_seen = self.user_dict[user_id]
        candidate = []
        for s, movie in self.movie_similary[movie_id]:
            if movie in user_seen:
                for idx, u in enumerate(self.group_dict[movie]):
                    if u == user_id:
                        candidate.append((movie, s, self.score_dict[movie][idx]))
                        break
        
        pred = sum(item[1] * (item[2] - self.user_mean[user_id]) for item in candidate) / sum(np.abs(item[1]) for item in candidate)
        return pred + self.user_mean[user_id]

df = pd.read_csv("C:\\vscode_project\\data_mining_homework\\data\\ml-latest-small\\ratings.csv")
df = df.sort_values("movieId")
cf = Collaborative_Filtering(100, 2)
cf.train(df)

Progress:  99.99%

In [26]:
def new_prodict(self, user_id, movie_id):
    user_seen = self.user_dict[user_id]
    candidate = []
    for s, movie in self.movie_similary[movie_id]:
        if movie in user_seen:
            for idx, u in enumerate(self.group_dict[movie]):
                if u == user_id:
                    candidate.append((movie, s, self.score_dict[movie][idx]))
                    break
    
    pred = sum(item[1] * (item[2] - self.user_mean[user_id]) for item in candidate) / sum(np.abs(item[1]) for item in candidate)
    return pred + self.user_mean[user_id]

cf.predict = new_prodict

In [58]:
import random
import numpy as np

def test_cf_on_5_users(cf):
    # ===== 1. 构建 user -> [(movie, rating), ...] =====
    user_ratings = {}

    for movie, users in cf.group_dict.items():
        scores = cf.score_dict[movie]
        for u, r in zip(users, scores):
            user_ratings.setdefault(u, []).append((movie, r))

    # 只保留评分数 >= 2 的用户（否则没法留一）
    valid_users = [u for u in user_ratings if len(user_ratings[u]) >= 2]

    # ===== 2. 随机选 5 个用户 =====
    test_users = random.sample(valid_users, 5)

    errors = []
    abs_errors = []

    for u in test_users:
        items = user_ratings[u]

        # 随机留一条评分
        test_movie, true_rating = random.choice(items)

        # ===== 3. 临时“遮蔽”该评分 =====
        # 从该 movie 的用户列表中移除 u
        idx = cf.group_dict[test_movie].index(u)

        backup_user = cf.group_dict[test_movie][idx]
        backup_score = cf.score_dict[test_movie][idx]

        cf.group_dict[test_movie].pop(idx)
        cf.score_dict[test_movie].pop(idx)

        try:
            # ===== 4. 预测 =====
            pred = cf.predict(cf,u, test_movie)

            # 如果你的 predict 可能返回 None
            if pred is None:
                continue

            errors.append((pred - true_rating) ** 2)
            abs_errors.append(abs(pred - true_rating))

            print(f"User {u}, Movie {test_movie}, True={true_rating:.1f}, Pred={pred:.3f}")

        finally:
            # ===== 5. 恢复数据 =====
            cf.group_dict[test_movie].insert(idx, backup_user)
            cf.score_dict[test_movie].insert(idx, backup_score)

    # ===== 6. 计算指标 =====
    rmse = np.sqrt(np.mean(errors))
    mae = np.mean(abs_errors)

    print("\n===== Evaluation Result =====")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE : {mae:.4f}")


test_cf_on_5_users(cf)

User 537, Movie 6888, True=5.0, Pred=4.260
User 511, Movie 105844, True=4.0, Pred=4.024
User 114, Movie 5816, True=4.0, Pred=3.194
User 299, Movie 3578, True=5.0, Pred=3.974
User 212, Movie 182823, True=4.0, Pred=3.777

===== Evaluation Result =====
RMSE: 0.6784
MAE : 0.5638


In [ ]:
print()